# 🎯 Pelatihan Model SmartCash

Notebook ini menjelaskan proses pelatihan model SmartCash sesuai dengan interface CLI.

## 📋 Daftar Isi
1. [Setup Environment](#setup)
2. [Konfigurasi Model](#konfigurasi)
3. [Persiapan Data](#data)
4. [Pelatihan Model](#training)
5. [Monitoring & Visualisasi](#monitoring)

## 1. Setup Environment <a id='setup'></a>

In [ ]:
import sys
import os
from pathlib import Path

# Add project root to path
project_root = Path().absolute().parent
sys.path.append(str(project_root))

# Import handlers & models
from smartcash.handlers.data_handler import DataHandler
from smartcash.handlers.training_handler import TrainingHandler
from smartcash.models.yolov5_model import YOLOv5Model
from smartcash.models.baseline import BaselineModel

## 2. Konfigurasi Model <a id='konfigurasi'></a>

In [ ]:
# Load config
import yaml
from dotenv import load_dotenv

load_dotenv()

with open(project_root / 'configs' / 'base_config.yaml') as f:
    config = yaml.safe_load(f)

### 2.1 Pilih Mode Deteksi

Pilih antara deteksi lapis tunggal atau banyak:

In [ ]:
# Pilih mode deteksi
MULTI_LAYER = True  # Set False untuk deteksi lapis tunggal

config['model']['multi_layer'] = MULTI_LAYER

### 2.2 Pilih Arsitektur Model

Pilih antara CSPDarknet atau EfficientNet-B4:

In [ ]:
# Pilih arsitektur
USE_EFFICIENT = True  # Set False untuk CSPDarknet

if USE_EFFICIENT:
    model = YOLOv5Model(config)
else:
    model = BaselineModel(config)

### 2.3 Konfigurasi Parameter Pelatihan

In [ ]:
# Set parameter pelatihan
training_config = {
    'batch_size': 32,
    'learning_rate': 0.001,
    'num_epochs': 100,
    'early_stopping': 10
}

config['training'].update(training_config)

## 3. Persiapan Data <a id='data'></a>

In [ ]:
# Load dataset
handler = DataHandler()
train_loader, val_loader = handler.get_data_loaders(
    batch_size=config['training']['batch_size']
)

## 4. Pelatihan Model <a id='training'></a>

In [ ]:
# Inisialisasi trainer
trainer = TrainingHandler(
    model=model,
    config=config,
    train_loader=train_loader,
    val_loader=val_loader
)

# Mulai pelatihan
history = trainer.train()

## 5. Monitoring & Visualisasi <a id='monitoring'></a>

In [ ]:
# Plot training history
trainer.plot_history(history)

# Save model
trainer.save_model(output_dir=project_root / 'outputs')